In [196]:
#Load the necessary Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
from typing import List
import warnings

In [197]:
# Load the dataset
data = pd.read_csv('train.csv')

In [198]:
# Separate features and target variable
X = data.drop('Cover_Type', axis=1)
y = data['Cover_Type']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [199]:
# Initialize classifiers
classifiers = [
    ('Linear Classification', LogisticRegression(max_iter=1000)),
    ('SVM', SVC()),
    ('KNN Classification', KNeighborsClassifier()),
    ('Decision Trees', DecisionTreeClassifier()),
    ('Adaboost', AdaBoostClassifier()),
    ('MLP', MLPClassifier(max_iter=1000))
]

In [200]:
# Train and evaluate classifiers
for name, classifier in classifiers:
    classifier.fit(X_train_scaled, y_train)
    y_pred = classifier.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{name}: Accuracy = {accuracy:.4f}')

Linear Classification: Accuracy = 0.7116
SVM: Accuracy = 0.7444
KNN Classification: Accuracy = 0.7738
Decision Trees: Accuracy = 0.8079
Adaboost: Accuracy = 0.3466
MLP: Accuracy = 0.8267


In [201]:
# Create an Ensemble classifier with top 3 and top 5 models using VotingClassifier
top_3_classifiers = [
    ('MLP', MLPClassifier()),
    ('KNN Classification', KNeighborsClassifier()),
    ('Decision Trees', DecisionTreeClassifier())
]

In [202]:
top_5_classifiers = classifiers[:5]

voting_classifier_3 = VotingClassifier(estimators=top_3_classifiers, voting='hard')
voting_classifier_5 = VotingClassifier(estimators=top_5_classifiers, voting='hard')

In [203]:
# Train and evaluate the ensemble classifiers
voting_classifier_3.fit(X_train_scaled, y_train)
voting_classifier_5.fit(X_train_scaled, y_train)

y_pred_3 = voting_classifier_3.predict(X_test_scaled)
y_pred_5 = voting_classifier_5.predict(X_test_scaled)

accuracy_3 = accuracy_score(y_test, y_pred_3)
accuracy_5 = accuracy_score(y_test, y_pred_5)

print(f'Ensemble Classifier (Top 3): Accuracy = {accuracy_3:.4f}')
print(f'Ensemble Classifier (Top 5): Accuracy = {accuracy_5:.4f}')

S:\Karim\Internship\Anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Ensemble Classifier (Top 3): Accuracy = 0.8356
Ensemble Classifier (Top 5): Accuracy = 0.7956


In [222]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
from typing import List, Union
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from pydantic import BaseModel
import pickle

In [223]:
# Load the trained models
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

In [224]:
# Train and save the MLP model
model_mlp = MLPClassifier(max_iter=1000)
model_mlp.fit(X_train_scaled, y_train)
joblib.dump(model_mlp, 'model_mlp.pkl')

['model_mlp.pkl']

In [225]:
# Train and save the KNN model
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train_scaled, y_train)
joblib.dump(model_dt, 'model_dt.pkl')

['model_dt.pkl']

In [226]:
# Train and save the ensemble classifier (voting_classifier_3)
model_ensemble = VotingClassifier(estimators=top_3_classifiers, voting='hard')
model_ensemble.fit(X_train_scaled, y_train)
joblib.dump(model_ensemble, 'model_ensemble.pkl')

S:\Karim\Internship\Anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['model_ensemble.pkl']

In [209]:
# Load the trained models 
model_mlp = joblib.load('model_mlp.pkl')
model_dt = joblib.load('model_dt.pkl')
model_ensemble = joblib.load('model_ensemble.pkl')

In [210]:
app = FastAPI()

In [211]:
class PredictionResponse(BaseModel):
    predicted_cover_type: int
    predictions: List[int]

In [213]:
# Define the root (index) endpoint
@app.get("/")
def read_root():
    return {"hello": "Welcome to the Forest Cover Type Prediction API"}

In [214]:
# Define the version endpoint
@app.get("/version")
def read_version():
    return {"version": "1.0"}

In [215]:
@app.post("/predict/mlp")
def predict_mlp(data: dict):
    try:
        features = np.array(list(data.values())).reshape(1, -1)
        predictions = model_mlp.predict(features)
        predicted_cover_type = int(predictions[0])
        return PredictionResponse(predicted_cover_type=predicted_cover_type, predictions=predictions.tolist())
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [216]:
@app.post("/predict/decision-trees")
def predict_decision_trees(data: dict):
    try:
        features = np.array(list(data.values())).reshape(1, -1)
        predictions = model_dt.predict(features)
        predicted_cover_type = int(predictions[0])
        return PredictionResponse(predicted_cover_type=predicted_cover_type, predictions=predictions.tolist())
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [218]:
@app.post("/predict/ensemble")
def predict_ensemble(input_data):
    try:
        features = np.array(list(input_data.dict().values())).reshape(1, -1)
        predictions = model_ensemble.predict(features)
        predicted_cover_type = int(predictions[0])
        return PredictionResponse(predicted_cover_type=predicted_cover_type, predictions=predictions.tolist())
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [220]:
import uvicorn

In [221]:
if __name__ == "__main__":
    uvicorn.run("app:app", host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop